# cs.py

In [1]:
import argparse
import os
from functools import reduce
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
# !conda install -c conda-forge cvxpy
import cvxpy as cp

In [3]:
dir_A = "datasets/{}/{}/".format('children', 'td')
files_A = ["{}{}".format(dir_A, filename) for filename in os.listdir(dir_A)]

In [4]:
dir_B = "datasets/{}/{}/".format('children', 'asd')
files_B = ["{}{}".format(dir_B, filename) for filename in os.listdir(dir_B)]

In [5]:
np.loadtxt(files_A[0])

array([[1., 1., 0., ..., 0., 1., 0.],
       [1., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 1.],
       [1., 0., 0., ..., 0., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.]])

In [6]:
# Split into training and testing samples
train_A, test_A = train_test_split(files_A, test_size=0.2, random_state=23)
train_B, test_B = train_test_split(files_B, test_size=0.2, random_state=23)

In [7]:
# Create and Write Summary Graphs
summary_A = reduce(lambda x,y:x+y, map(lambda file: np.loadtxt(file), train_A))/len(train_A)
summary_B = reduce(lambda x,y:x+y, map(lambda file: np.loadtxt(file), train_B))/len(train_B)

In [8]:
summary_A

array([[1.        , 0.85365854, 0.31707317, ..., 0.07317073, 0.07317073,
        0.07317073],
       [0.85365854, 1.        , 0.12195122, ..., 0.09756098, 0.02439024,
        0.14634146],
       [0.31707317, 0.12195122, 1.        , ..., 0.04878049, 0.02439024,
        0.        ],
       ...,
       [0.07317073, 0.09756098, 0.04878049, ..., 1.        , 0.87804878,
        0.56097561],
       [0.07317073, 0.02439024, 0.02439024, ..., 0.87804878, 1.        ,
        0.82926829],
       [0.07317073, 0.14634146, 0.        , ..., 0.56097561, 0.82926829,
        1.        ]])

In [9]:
summary_B

array([[1.        , 0.82051282, 0.25641026, ..., 0.05128205, 0.07692308,
        0.15384615],
       [0.82051282, 1.        , 0.05128205, ..., 0.15384615, 0.07692308,
        0.05128205],
       [0.25641026, 0.05128205, 1.        , ..., 0.07692308, 0.07692308,
        0.05128205],
       ...,
       [0.05128205, 0.15384615, 0.07692308, ..., 1.        , 0.84615385,
        0.58974359],
       [0.07692308, 0.07692308, 0.07692308, ..., 0.84615385, 1.        ,
        0.74358974],
       [0.15384615, 0.05128205, 0.05128205, ..., 0.58974359, 0.74358974,
        1.        ]])

In [10]:
diff_net = summary_A - summary_B

In [11]:
diff_net.dtype

dtype('float64')

In [12]:
np.save('rewrite/deff_net', diff_net)

# densdp.py

In [13]:
import networkx as nx
# import oqc_sdp
import sys
import os
import numpy as np
import scipy.io
import subprocess
import time
# from greedy_oqc import localSearchNegativeOQC
import pickle

In [14]:
G = nx.Graph()
graphfile = 'icdm16-egoscan/net.txt'
with open(graphfile) as f:
    for line in f:
#         print(line)
        tokens = line.strip().split()
        u = int(tokens[0])
        v = int(tokens[1])
        w = float(tokens[2])
        G.add_edge(u, v, weight=w)

In [15]:
G1 = nx.from_numpy_array(diff_net)

In [16]:
nx.is_isomorphic(G1, G)

False

In [17]:
print ("Loaded graph with %s nodes and %s edges" % (len(G), G.number_of_edges()))

Loaded graph with 116 nodes and 6670 edges


In [18]:
X = scipy.loadtxt('icdm16-egoscan/net.txt')

<ipython-input-18-aa63a037ee1e>:1: DeprecationWarning: scipy.loadtxt is deprecated and will be removed in SciPy 2.0.0, use numpy.loadtxt instead
  X = scipy.loadtxt('icdm16-egoscan/net.txt')


In [19]:
X.shape

(6670, 3)

In [20]:
X

array([[ 1.00000000e+00,  0.00000000e+00, -7.45682889e-03],
       [ 2.00000000e+00,  0.00000000e+00,  2.74725275e-03],
       [ 2.00000000e+00,  1.00000000e+00,  7.45682889e-02],
       ...,
       [ 1.15000000e+02,  1.12000000e+02, -1.88383046e-02],
       [ 1.15000000e+02,  1.13000000e+02, -5.33751962e-02],
       [ 1.15000000e+02,  1.14000000e+02,  5.14128728e-02]])

# oqc_sdp.py

In [21]:
import networkx as nx
import numpy as np

In [22]:
def _make_coefficient_matrices(A, weight='weight'):
    N = len(A)
    # w is the matrix of coefficients for 
    # the objective function
    w = np.zeros((N + 1, N + 1))
    # the (0, j)th entry of w is the sum of column j
    w_0j = np.zeros(N + 1)
    w_0j[1:] = A.sum(axis=0)
    w[0, :] = w_0j
    # the (i, 0)th entry of w is the sum of row i
    w_i0 = np.zeros(N + 1)
    w_i0[1:] = A.sum(axis=1).T
    w[:, 0] = w_i0
    # the rest of w (i.e., entries (i, j), such that i != 0 and j != 0)
    # are just the adjacency matrix of G
    w[1:, 1:] = A
    # diagonal elements should be all zero (no self-loops)
    np.fill_diagonal(w, 0.)
    
    # d is the matrix of coefficients for constraint (1)
    # For constraint (1), we want each edge in the final solution to
    # contribute a weight of 1
    # We can think of d as being the corresponding matrix w
    # for a complete graph (i.e., each edge has weight 1)
    d = np.ones((N + 1, N + 1))
    # the (0, j)th entry of d is the sum of column j (i.e., N - 1)
    d_0j = np.ones(N + 1) * (N - 1)
    d[0, :] = d_0j
    # same for the (i, 0)th entries
    d_i0 = np.ones(N + 1) * (N - 1)
    d[:, 0] = d_i0
    np.fill_diagonal(d, 0.)

    return w, d

In [23]:
w, d = _make_coefficient_matrices(diff_net)

In [24]:
w, d

(array([[ 0.        ,  2.39587242,  0.26579112, ...,  0.13946216,
         -1.63101939, -2.81113196],
        [ 2.39587242,  0.        ,  0.03314572, ...,  0.02188868,
         -0.00375235, -0.08067542],
        [ 0.26579112,  0.03314572,  0.        , ..., -0.05628518,
         -0.05253283,  0.09505941],
        ...,
        [ 0.13946216,  0.02188868, -0.05628518, ...,  0.        ,
          0.03189493, -0.02876798],
        [-1.63101939, -0.00375235, -0.05253283, ...,  0.03189493,
          0.        ,  0.08567855],
        [-2.81113196, -0.08067542,  0.09505941, ..., -0.02876798,
          0.08567855,  0.        ]]),
 array([[  0., 115., 115., ..., 115., 115., 115.],
        [115.,   0.,   1., ...,   1.,   1.,   1.],
        [115.,   1.,   0., ...,   1.,   1.,   1.],
        ...,
        [115.,   1.,   1., ...,   0.,   1.,   1.],
        [115.,   1.,   1., ...,   1.,   0.,   1.],
        [115.,   1.,   1., ...,   1.,   1.,   0.]]))

In [25]:
alpha = 0.05

In [26]:
P = np.matrix(w - alpha * d)

In [27]:
P1 = w - alpha * d

In [28]:
(P == P1).sum()

13689

In [29]:
P.shape

(117, 117)